# 架構圖 (也可看檔案夾裡的原圖)

#### 1. 主要分為兩個部分，上半部為資料搜集&處理，下半部為採用向量方式，比對文字的相似度
#### 2. 在判決書向量計算上，會先存起來，當query文字進來，只要計算query向量，並進行向量相似度計算，減少每次搜索運算量。

![test](data_flow_judicial.jpg)

# 此腳本為資料蒐集與處理

In [4]:
from pathlib import Path
import requests
import pickle

import pandas as pd

#### step1. get historical judicial id

In [5]:
root_path = Path('/Users/zoe/Desktop/Kevin_project/judicial_analysis')
input_path = root_path / 'data/108/upload10812'

/Users/zoe/Desktop/Kevin_project/judicial_analysis/108/upload10812


In [6]:
cols = [
 '控制碼'
,'法院別'
,'案號-年'
,'案號-字別'
,'案號-號'
,'案由'
,'法官名1'
,'法官名2'
,'法官名3'
,'原審(確定)法院'
,'原審(確定)案號-年'
,'原審(確定)案號-字別'
,'原審(確定)案號-號'
,'原告/上訴人是否有律師訴訟代理'
,'被告/被上訴人是否有律師訴訟代理'
,'全案終結日期-年'
,'全案終結日期-月'
,'全案終結日期-日'
,'全案終結情形1'
,'全案終結情形2'
,'全案終結情形3'
,'全案終結情形4'
,'得上訴'
,'不得上訴'
,'得抗告'
,'不得抗告'
,'被告是否反訴'
,'反訴終結情形'
,'訴訟標的類別'
,'訴訟標的金額'
,'裁判內容出現「消除對婦女一切形式歧視公約」'
,'裁判內容出現「兒童權利公約」'
,'裁判內容出現「兩人權公約」'
,'裁判內容出現「身心障礙者權利公約」'
,'被請求機關之機關類別'
,'賠償類別'
,'賠償機關如為法院時之義務歸屬'
,'公職類別'
,'選舉類別'
,'原告身分'
,'被告身分'
,'上訴或抗告人別'
,'裁判書ID'
]

In [7]:
all_df = pd.DataFrame()

for file in input_path.glob('*/*民事訴訟.txt'):
    try:
        df = pd.read_csv(file, encoding='big5hkscs', sep='!', names=cols, index_col=False, lineterminator='#')
    except Exception as e:
        print(e)
        print(file, ' failed!')
        continue

    df.控制碼 = df.控制碼.replace('\r\n', '', regex=True)
    all_df = all_df.append(df)


'big5hkscs' codec can't decode byte 0xa4 in position 23246: illegal multibyte sequence
/Users/zoe/Desktop/Kevin_project/judicial_analysis/108/upload10812/高雄簡易庭民事/10812.KSEV.民事訴訟.txt  failed!


In [83]:
all_df.to_excel('data/民事_hist.xlsx')

#### step2. get judical api token & content

In [6]:
# account/pwd should be yml config file to make it safe
# 因考慮本院網路頻寬及系統負擔，並避免損及多數一般使用者權益，本API提供服務時間為每日凌晨0時至6時止，其餘時間恕不提供服務。
rs = requests.post('https://data.judicial.gov.tw/jdg/api/Auth', data={'user':'youraccount', 'password':'yourpwd'})
api_key = rs.json()['token']

In [9]:
#about 2 hours for 15,000 requests
judicial_json = []


for idx, jid in enumerate(all_df[all_df['裁判書ID'].notna()]['裁判書ID']):
    if idx % 100 == 0:
        print(idx)
    rs = requests.post('https://data.judicial.gov.tw/jdg/api/JDoc', data={'token':api_key, 'j':jid})
    judicial_json.append(rs.json())

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900


In [10]:
with open('all_judicial.pickle', 'wb') as f:
    pickle.dump(judicial_json, f)

In [19]:
# 有車禍的判決數量
[ 1 for x in judicial_json if x.get('JFULLX') and '車禍' in x['JFULLX']['JFULLCONTENT']].__len__()

786